In [ ]:
# 04_business_insights.ipynb

# ---------------------------------------------
# Recomendaciones de Negocio - Open Banking Challenge
# Autor: Pablo Flores
# ---------------------------------------------


In [ ]:
# 1. Librerías necesarias
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import joblib


In [ ]:
# 2. Cargar datos y modelo
df = pd.read_csv("data/processed/final_dataset.csv")
model = joblib.load("outputs/models/xgboost_model.pkl")  # Cambiar según el mejor modelo guardado


In [ ]:
# 3. Predecir probabilidades sobre todo el dataset
X = df.drop(columns=["has_insurance"])
df["pred_proba"] = model.predict_proba(X)[:, 1]


In [ ]:
# 4. Distribución de probabilidades
sns.histplot(df["pred_proba"], bins=20, kde=True)
plt.title("Distribución de probabilidad de contratar seguro")
plt.xlabel("Probabilidad estimada")
plt.ylabel("Número de clientes")
plt.show()

In [ ]:
# 5. Top clientes para campaña
top_clients = df.sort_values("pred_proba", ascending=False).head(10)
top_clients[["pred_proba"] + [col for col in df.columns if col.startswith("income_range") or col.startswith("risk_profile")]]


In [ ]:
# 6. Segmentación de clientes
df["segment"] = pd.cut(
    df["pred_proba"],
    bins=[0, 0.3, 0.6, 1],
    labels=["Bajo interés", "Interés medio", "Alto interés"]
)

segment_counts = df["segment"].value_counts().sort_index()
sns.barplot(x=segment_counts.index, y=segment_counts.values)
plt.title("Segmentación de clientes por propensión")
plt.ylabel("Número de clientes")
plt.xlabel("Segmento")
plt.show()

In [ ]:
# 7. Análisis cruzado con perfil de riesgo
seg_cross = pd.crosstab(df["segment"], df.filter(like="risk_profile_").idxmax(axis=1))
seg_cross.plot(kind="bar", stacked=True, colormap="Set2")
plt.title("Distribución del perfil de riesgo dentro de cada segmento")
plt.ylabel("Clientes")
plt.xlabel("Segmento de Propensión")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:


# 📌 8. Conclusiones y recomendaciones
from IPython.display import Markdown
Markdown("""
## Recomendaciones de Negocio

- **Segmento de alto interés (60-100%)**: Representa el X% de la base. Este grupo debe ser **prioridad 1** para campañas dirigidas, especialmente si ya tienen productos de inversión o ahorro.
- **Perfiles 'moderate' con ingresos medios-altos** dominan en el segmento alto de probabilidad.
- La categoría de gasto más frecuente puede ser un excelente disparador para personalizar ofertas (por ejemplo, viajes → seguro de viaje).
- Sugerencia: ofrecer un seguro empaquetado o con descuento a quienes ya tengan 2+ productos financieros.

## Próximos pasos

- Integrar el modelo en una API o dashboard para uso del equipo comercial.
- Hacer seguimiento de la efectividad de las campañas y reentrenar el modelo periódicamente.
""")
